## Notebook 5: Análisis de Casos de Estudio Específicos

**Objetivo:** Analizar y visualizar productos individuales para ambas vertientes (unidades y guaraníes).

**Fases:**
1.  **Configuración:** Importar librerías y funciones.
2.  **Carga de Datos:** Cargar el dataset de ventas consolidado.
3.  **Bucle de Análisis de Casos:** Iterar sobre 'cantidad' y 'monto_venta'.
    a. Cargar los resultados del modelo para la vertiente correspondiente.
    b. Definir los productos de estudio.
    c. Generar y guardar los gráficos.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# =============================================================================
# CELDA 1: IMPORTACIONES Y CONFIGURACIÓN
# =============================================================================

import pandas as pd
import numpy as np
import os
import sys
import logging
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown
from dbfread import DBF 

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
print("Librerías importadas y logging configurado.")

# --- Configuración de Matplotlib para la Tesis ---
try:
    plt.rcParams.update({
        "font.family": "serif",
        "font.serif": ["Times New Roman"],
        "font.size": 12
    })
    print("Configuración de Matplotlib para 'Times New Roman' aplicada.")
except:
    print("Advertencia: No se pudo configurar la fuente 'Times New Roman'. Usando 'serif' por defecto.")

Librerías importadas y logging configurado.
Configuración de Matplotlib para 'Times New Roman' aplicada.


In [3]:
# =============================================================================
# CELDA 2: PARÁMETROS (¡Etiquetar esta celda como 'parameters' en Jupyter!)
# =============================================================================
# Papermill sobrescribirá estos valores
BASE_DIR = '..'
target_variable = 'all' # 'cantidad', 'monto_venta', o 'all'
datasets_to_run = ['F_48_NP'] # ¡ENFOCADO! Solo usamos F_48_NP
models_to_run = ['Lineal', 'Cuadrático', 'KNN', 'SARIMA']
metrics_to_calculate = ['R2', 'MAE', 'RMSE', 'MASE', 'ME']
maxiter = 50 # No se usa aquí

# --- ¡Parámetros Clave para ESTE Notebook! ---
MAIN_METRIC = 'MASE' 
METRIC_THRESHOLD = 1.0

print(f"Análisis de Casos de Estudio configurado para: {datasets_to_run}")
print(f"Métrica principal de selección: {MAIN_METRIC}")

Análisis de Casos de Estudio configurado para: ['F_48_NP']
Métrica principal de selección: MASE


In [4]:
# =============================================================================
# CELDA 3: CONFIGURACIÓN DE RUTAS E IMPORTACIÓN DE UTILS
# =============================================================================
# Añadir utils al path
utils_path = os.path.join(BASE_DIR, '03_Code', 'utils')
if utils_path not in sys.path:
    sys.path.append(utils_path)

# Importar la función *actualizada* de nuestro motor
try:
    from utils.utils import analizar_y_graficar_casos_de_estudio
    print("Función 'analizar_y_graficar_casos_de_estudio' importada desde utils.py.")
except ImportError:
    print("ERROR: No se pudo importar la función desde utils.py. Asegúrate de que el archivo esté en la ruta correcta.")

# --- Directorios ---
PROCESSED_DATA_DIR = os.path.join(BASE_DIR, '02_Data', '02_Processed') # ¡Leer datos limpios!
RESULTS_DIR = os.path.join(BASE_DIR, '04_Resultados')
TABLES_INPUT_DIR = os.path.join(RESULTS_DIR, '01_Tablas_Detalladas') 
FIGURES_OUTPUT_DIR = os.path.join(RESULTS_DIR, '02_Figuras', 'casos_de_estudio')

os.makedirs(FIGURES_OUTPUT_DIR, exist_ok=True)

logging.info(f"Directorio de Datos (Lectura): {PROCESSED_DATA_DIR}")
logging.info(f"Directorio de Resultados (Lectura): {TABLES_INPUT_DIR}")
logging.info(f"Directorio de Figuras (Escritura): {FIGURES_OUTPUT_DIR}")

2025-11-11 15:30:34,688 - INFO - Directorio de Datos (Lectura): ..\02_Data\02_Processed
2025-11-11 15:30:34,689 - INFO - Directorio de Resultados (Lectura): ..\04_Resultados\01_Tablas_Detalladas
2025-11-11 15:30:34,690 - INFO - Directorio de Figuras (Escritura): ..\04_Resultados\02_Figuras\casos_de_estudio


Función 'analizar_y_graficar_casos_de_estudio' importada desde utils.py.


In [5]:
# =============================================================================
# CELDA 4: BUCLE PRINCIPAL DE ANÁLISIS (Versión Final - 7 Casos Dirigidos)
# =============================================================================

# Determinar los targets a procesar
if isinstance(target_variable, list):
    targets_a_procesar = target_variable
elif target_variable == 'all':
    targets_a_procesar = ['cantidad', 'monto_venta']
else:
    targets_a_procesar = [target_variable]

# Asumimos que solo analizamos el primer dataset de la lista (F_48_NP)
dataset_name = datasets_to_run[0]

for target in targets_a_procesar:
    display(Markdown(f"# Casos de Estudio para: {target.upper()} (Dataset: {dataset_name})"))
    
    # --- 1. Cargar los DATOS de ventas (¡LOS DATOS LIMPIOS!) ---
    data_file_path = os.path.join(PROCESSED_DATA_DIR, target, f'ventas_{dataset_name}.csv')
    try:
        df_ventas_limpias = pd.read_csv(data_file_path, parse_dates=['mes'], dtype={'codigo_producto': str})
        logging.info(f"Datos de ventas '{dataset_name}' ({target}) cargados. (Shape: {df_ventas_limpias.shape})")
    except FileNotFoundError:
        logging.error(f"¡ERROR CRÍTICO! No se encontró el archivo de datos: {data_file_path}")
        continue 
        
    # --- 2. Cargar los RESULTADOS del modelo --
    results_file_path = os.path.join(TABLES_INPUT_DIR, target, f'resultados_detallados_{dataset_name}_{target}_CV.csv')
    try:
        df_resultados = pd.read_csv(results_file_path, dtype={'codigo_producto': str})
        logging.info(f"Resultados del modelo '{dataset_name}' ({target}) cargados. (Shape: {df_resultados.shape})")
    except FileNotFoundError:
        logging.error(f"¡ERROR CRÍTICO! No se encontró el archivo de resultados: {results_file_path}")
        continue

    # --- 3. Lógica "Auditoría Visual": Selección Dirigida de Casos (Versión Final) ---
    display(Markdown(f"## 3.1. Selección de Casos de Estudio (Muestreo Dirigido)"))
    
    col_ganador = f'modelo_ganador_{MAIN_METRIC}'
    col_valor = f'{MAIN_METRIC}_ganador'
    
    codigos_a_graficar = []

    if target == 'cantidad':
        # Casos de Éxito (MASE < 1.0)
        caso_lineal = '33139'     # MECHA TERRAX HSS 3MM (MASE 0.41)
        caso_cuad = '32001'       # FICHA HEMBRA SCHUKO VIVION NG (MASE 0.47)
        caso_sarima_ciclico = '33512' # (LA ESTRELLA CÍCLICA) PICO TOMA ELEKTRON (MASE 0.29)
        caso_sarima_tendencia = '70863' # (LA ESTRELLA DE TENDENCIA) CARTEL LED RIO (MASE 0.37)
        caso_knn = '31137'        # CAÑO CORRUGADO NJ 1" TIGRE (MASE 0.39)
        
        # Casos de Fallo (MASE >= 1.0)
        caso_fail_knn = '31564'   # CORTA HIERRO FAMASTIL 10" (MASE 1.05)
        caso_fail_sarima = '31699' # DESTORN TRAM PLANO 3/16X4" (MASE 1.01)

        codigos_a_graficar = [
            caso_lineal, caso_cuad, caso_sarima_ciclico, caso_sarima_tendencia, caso_knn,
            caso_fail_knn, caso_fail_sarima
        ]
    
    elif target == 'monto_venta':
        # Usamos los mismos SKUs para el anexo
        codigos_a_graficar = [
            '33139', '32001', '33512', '70863', '31137',
            '31564', '31699'
        ]
        print(f"Utilizando los mismos SKUs de 'cantidad' para el análisis de 'monto_venta'.")

    
    print(f"Se generarán {len(codigos_a_graficar)} gráficos de auditoría visual (Casos dirigidos).")

    # --- 4. Generar Gráficos (Llamando a utils.py) ---
    display(Markdown(f"## 3.2. Gráficos de Casos de Estudio (Auditoría)"))
    
    output_figs_subdir = os.path.join(FIGURES_OUTPUT_DIR, target)
    os.makedirs(output_figs_subdir, exist_ok=True)
    
    analizar_y_graficar_casos_de_estudio(
        codigos_productos=codigos_a_graficar,
        df_ventas_completo=df_ventas_limpias,
        df_resultados=df_resultados,
        figures_dir=output_figs_subdir,
        target_column=target,
        winning_col=col_ganador,
        metric_col=col_valor,
        models_available=models_to_run
    )

# Casos de Estudio para: CANTIDAD (Dataset: F_48_NP)

2025-11-11 15:30:35,719 - INFO - Datos de ventas 'F_48_NP' (cantidad) cargados. (Shape: (78595, 6))
2025-11-11 15:30:35,733 - INFO - Resultados del modelo 'F_48_NP' (cantidad) cargados. (Shape: (1044, 35))


## 3.1. Selección de Casos de Estudio (Muestreo Dirigido)

Se generarán 7 gráficos de auditoría visual (Casos dirigidos).


## 3.2. Gráficos de Casos de Estudio (Auditoría)

...

   - ..\04_Resultados\02_Figuras\casos_de_estudio\monto_venta\monto_venta_MASE_0.39_KNN_31137_CAÑO_CORRUGADO_NJ__1_TIGRE_32M.png
   - ..\04_Resultados\02_Figuras\casos_de_estudio\monto_venta\monto_venta_MASE_0.39_KNN_31137_CAÑO_CORRUGADO_NJ__1_TIGRE_32M.eps
 - Guardando gráficos para 'CORTA HIERRO FAMASTIL 10" CHATO 167-10"' (31564):


2025-11-11 15:30:52,163 - WARNING - The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


   - ..\04_Resultados\02_Figuras\casos_de_estudio\monto_venta\monto_venta_MASE_1.21_KNN_31564_CORTA_HIERRO_FAMASTIL_10_CHATO.png
   - ..\04_Resultados\02_Figuras\casos_de_estudio\monto_venta\monto_venta_MASE_1.21_KNN_31564_CORTA_HIERRO_FAMASTIL_10_CHATO.eps
 - Guardando gráficos para 'DESTORN TRAM PLANO 3/16X4"' (31699):


2025-11-11 15:30:52,579 - WARNING - The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


   - ..\04_Resultados\02_Figuras\casos_de_estudio\monto_venta\monto_venta_MASE_1.05_Lineal_31699_DESTORN_TRAM_PLANO_316X4.png
   - ..\04_Resultados\02_Figuras\casos_de_estudio\monto_venta\monto_venta_MASE_1.05_Lineal_31699_DESTORN_TRAM_PLANO_316X4.eps


In [6]:
# =============================================================================
# CELDA 9: FIN DEL NOTEBOOK
# =============================================================================
print("\n" + "="*50)
print("FIN DEL NOTEBOOK 05 - ANÁLISIS DE CASOS DE ESTUDIO")
print("="*50 + "\n")


FIN DEL NOTEBOOK 05 - ANÁLISIS DE CASOS DE ESTUDIO

